In [1]:
pip install geopandas

In [2]:
pip install plotly

In [3]:
pip install scikit-learn

Note: you may need to restart the kernel to use updated packages.


In [4]:
import pandas as pd
import plotly.offline as pyo
import plotly.graph_objs as go
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
import geopandas as gpd
from shapely.geometry import Point, LineString
import warnings

In [58]:
airport = pd.read_csv('flight_data/airports.csv')
routes = pd.read_csv('flight_data/routes.csv')
print(airports)
print(routes)

      index  Airport ID                                         Name  \
0         0           1                               Goroka Airport   
1         1           2                               Madang Airport   
2         2           3                 Mount Hagen Kagamuga Airport   
3         3           4                               Nadzab Airport   
4         4           5  Port Moresby Jacksons International Airport   
...     ...         ...                                          ...   
7693   7693       14106                          Rogachyovo Air Base   
7694   7694       14107                        Ulan-Ude East Airport   
7695   7695       14108                         Krechevitsy Air Base   
7696   7696       14109                  Desierto de Atacama Airport   
7697   7697       14110                           Melitopol Air Base   

              City           Country IATA  ICAO   Latitude   Longitude  \
0           Goroka  Papua New Guinea  GKA  AYGA  -6.081690  1

In [59]:
airport.columns

Index(['index', 'Airport ID', 'Name', 'City', 'Country', 'IATA', 'ICAO',
       'Latitude', 'Longitude', 'Altitude', 'Timezone', 'DST',
       'Tz database time zone', 'Type', 'Source'],
      dtype='object')

In [60]:
routes.columns

Index(['index', 'Airline', 'Airline ID', 'Source airport', 'Source airport ID',
       'Destination airport', 'Destination airport ID', 'Codeshare', 'Stops',
       'Equipment'],
      dtype='object')

In [ ]:
def plot_flights_from_country(country, internal=False):
    # Filter routes based on the country
    country_routes = routes[(routes['Source airport'].isin(airports[airports['Country'] == country]['IATA']))]
    
    # Optionally, filter internal flights
    if internal:
        country_airports = airports[airports['Country'] == country]['IATA']
        country_routes = country_routes[country_routes['Destination airport'].isin(country_airports)]
    
    # Create Plotly traces for flight paths
    flight_paths = []
    for i, row in country_routes.iterrows():
        origin = airports.loc[airports['IATA'] == row['Source airport']].iloc[0]
        destination = airports.loc[airports['IATA'] == row['Destination airport']].iloc[0]

        flight_paths.append(
            go.Scattergeo(
                locationmode='ISO-3',
                lon=[float(origin['Longitude']), float(destination['Longitude'])],
                lat=[float(origin['Latitude']), float(destination['Latitude'])],
                mode='lines',
                line=dict(width=2, color='blue'),
                opacity=0.7,
                name=f"{origin['IATA']} to {destination['IATA']}"
            )
        )

    # Set layout for the map
    layout = go.Layout(
        title='Internal flights' if internal else f"All flights from {country}",
        showlegend=True,
        geo=dict(
            scope='world',
            projection_type='natural earth',
            showland=True,
            landcolor='rgb(243, 243, 243)',
            countrycolor='rgb(204, 204, 204)',
        ),
    )

    # Create the figure and add the traces
    fig = go.Figure(data=flight_paths, layout=layout)
    fig.show()

# Example usage:
plot_flights_from_country('United States', internal=True)
plot_flights_from_country('United States', internal=False)
